In [2]:
import dlc_bci as bci

In [3]:
train_input , train_target = bci.load(root = './data_bci', one_khz=True)

print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])


In [4]:
test_input, test_target = bci.load(root = './data_bci', train=False, one_khz=True)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [5]:
train_input[0].shape

torch.Size([28, 500])

In [6]:
import torch
from torch.autograd import Variable
from torch.nn import functional as F

In [7]:
train_input.shape

torch.Size([316, 28, 500])

### 0. Data augmentation

In [31]:
# using the noise addition method to augment the EEG samples

In [8]:
noise_intensity =  [0.001, 0.01, 0.02, 0.1, 0.2, 0.5]

In [39]:
def add_centered_gaussian_noise(mean, std):
    _std = torch.from_numpy(np.full(mean.shape, std))
    _std = _std.type(torch.FloatTensor)
    return torch.normal(mean, _std)

In [60]:
train_augmented = []
for std in noise_intensity:
    for i in range(train_input.shape[0]):
        train_augmented.append(add_centered_gaussian_noise(train_input[i], std))

In [62]:
train_augmented = list(map(lambda t: torch.unsqueeze(t, 0), train_augmented))
train_augmented[0].shape

torch.Size([1, 28, 500])

In [63]:
train_augmented = torch.cat(train_augmented, 0)
print(type(train_augmented), train_augmented.shape)

<class 'torch.FloatTensor'> torch.Size([1896, 28, 500])


In [65]:
train_augmented = torch.cat((train_augmented, train_input), 0)
print(type(train_augmented), train_augmented.shape)

<class 'torch.FloatTensor'> torch.Size([2212, 28, 500])


<h3>Stardardize the data</h3>

<h3>1. normalize data channel-wise</h3>

In [21]:
right_indices = train_target.nonzero()

In [22]:
left_indices = (train_target == 0).nonzero()

In [23]:
# reshape from [157, 1] to [157]
right_indices = right_indices.view(-1)
left_indices = left_indices.view(-1)

In [24]:
right_data = torch.index_select(train_input, 0, right_indices)
left_data = torch.index_select(train_input, 0, left_indices)

print(right_data.shape, left_data.shape)

torch.Size([157, 28, 500]) torch.Size([159, 28, 500])


In [28]:
right_data[0]


 135.7000  132.0000  126.9000  ...    82.7000   88.2000   92.5000
  16.9000   13.0000    8.7000  ...   -30.1000  -20.7000  -14.6000
  23.9000   24.3000   20.1000  ...   -15.7000   -8.7000   -1.5000
             ...                ⋱                ...             
 -18.8000  -23.0000  -27.4000  ...   -16.5000  -16.9000  -16.0000
   6.1000    0.5000   -5.5000  ...    14.3000   13.8000   14.6000
 -27.5000  -31.8000  -36.7000  ...   -16.6000  -17.0000  -16.1000
[torch.FloatTensor of size 28x500]

In [29]:
rd_0 = torch.nn.functional.normalize(right_data[0], p=2, dim=0)

In [30]:
rd_0


 0.6434  0.6293  0.6041  ...   0.4748  0.5113  0.5391
 0.0801  0.0620  0.0414  ...  -0.1728 -0.1200 -0.0851
 0.1133  0.1158  0.0957  ...  -0.0901 -0.0504 -0.0087
          ...             ⋱             ...          
-0.0891 -0.1096 -0.1304  ...  -0.0947 -0.0980 -0.0933
 0.0289  0.0024 -0.0262  ...   0.0821  0.0800  0.0851
-0.1304 -0.1516 -0.1747  ...  -0.0953 -0.0986 -0.0938
[torch.FloatTensor of size 28x500]

In [122]:
right_data = torch.nn.functional.normalize(right_data, p=2, dim=0)
left_data = torch.nn.functional.normalize(left_data, p=2, dim=0)

In [123]:
class Net(torch.nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(28*500, nb_hidden)
        self.fc2 = torch.nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [124]:
def convert_to_one_hot_labels(input, target):
    tmp = input.new(target.size(0), target.max() + 1).fill_(-1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [125]:
train_target = convert_to_one_hot_labels(train_input, train_target)

In [126]:
# merge back right and left data into one data-set
train_input_norm = torch.FloatTensor(train_input.shape).fill_(0)
print(train_input_norm.shape)

torch.Size([316, 28, 500])


In [127]:
right_data.shape

torch.Size([157, 28, 500])

In [128]:
for i, ind in enumerate(right_indices):
    train_input_norm[ind] = right_data[i]
    
for i, ind in enumerate(left_indices):
    train_input_norm[ind] = left_data[i]

In [106]:
def train_model(model, train_input, train_target, mini_batch_size):
    criterion = torch.nn.MSELoss()
    eta = 1e-1

    for e in range(0, 25):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.data[0]
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        print(e, sum_loss)

In [107]:
def compute_nb_errors(model, input, target, mini_batch_size):

    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.data.max(1)
        for k in range(0, mini_batch_size):
            if target.data[b + k, predicted_classes[k]] < 0:
                nb_errors = nb_errors + 1

    return nb_errors

In [136]:
mini_batch_size = 4

In [130]:
test_input_norm = torch.nn.functional.normalize(test_input, p=2, dim=0)
test_target = convert_to_one_hot_labels(test_input, test_target)

In [131]:
train_input_norm, train_target = Variable(train_input_norm.view(-1,28*500)), Variable(train_target)
test_input_norm, test_target = Variable(test_input_norm.view(-1, 28*500)), Variable(test_target)

In [138]:
for k in range(0, 10):
    model = Net(200)
    train_model(model, train_input_norm, train_target, mini_batch_size) 
    nb_test_errors = compute_nb_errors(model, test_input_norm, test_target, mini_batch_size)
    print('test error Net {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                       nb_test_errors, test_input.size(0)))

0 81.52663904428482
1 80.71993315219879
2 80.66652083396912
3 80.78459882736206
4 80.60236662626266
5 80.72067052125931
6 80.97601813077927
7 80.31178167462349
8 81.70998400449753
9 81.9891117811203
10 81.01279896497726
11 79.6910138130188
12 81.20642054080963
13 80.0032406449318
14 79.58367276191711
15 80.64332780241966
16 79.27101674675941
17 76.59550747275352
18 81.558689057827
19 80.14358332753181
20 81.8133752644062
21 79.12926334142685
22 77.43111741542816
23 88.38135060667992
24 75.57860860228539
test error Net 51.00%% 51/100
0 81.56964349746704
1 80.89191764593124
2 80.72016829252243
3 80.80047136545181
4 80.919488966465
5 81.16407507658005
6 81.3935235440731
7 81.81209570169449
8 79.88167759776115
9 82.98465520143509
10 80.47977316379547
11 80.07123726606369
12 80.88505584001541
13 80.94243907928467
14 78.51867485046387
15 78.17379796504974
16 80.89411211013794
17 79.40369948744774
18 79.01745881140232
19 79.12541987001896
20 75.85671111941338
21 74.25923399627209
22 77.722254